In [1]:
# Print out the orders


In [2]:
# This is the file that was downloaded from 
order_form_entries = r"https://newmarketchurch.breezechms.com/forms/entries/557986"
print("Download as Excel from {orders}, and paste path into weekly_orders below.".format(orders = order_form_entries))

Download as Excel from https://newmarketchurch.breezechms.com/forms/entries/557986, and paste path into weekly_orders below.


In [3]:
import os
from pathlib import Path
from datetime import datetime

# This is the file that was downloaded from https://newmarketchurch.breezechms.com/forms/entries/557986
weekly_orders = r"C:\Users\ralph\Downloads\PANTRY-ORDER-FORM-111623 (1).xlsx"
title_date = '11/16/2023'


last_run = '10/17/2023 15:00'
current_run = datetime.now()
runmode = 'all'
# runmode = 'update'

# The number of labels to print for each shopper
number_of_labels = 6

# Get the exported filename for use in the report.
filename = Path(weekly_orders)
name = filename.name.replace(filename.suffix, '', -1)

filename = 'NCC Food Pantry Orders' 
title = filename + ' ' + title_date

# This file is created by this script and is to be imported at the same web page.
upload_file = os.path.join(os.path.dirname(weekly_orders), filename +'.html')
label_file = os.path.join(os.path.dirname(weekly_orders), filename +' labels.html')
# upload_file

In [4]:
import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
allorders = pd.read_excel(weekly_orders)
received = len(allorders.index)
print('{count} orders in input.'.format(count = received))

35 orders in input.


In [5]:
from datetime import datetime, timedelta

modes = {
    'all': 'All orders received this week',
    'update': 'All orders received since {date}'.format(date = last_run),
    'special': 'Scpecial criteria were used.'
    }

# Only process recent orders.
if runmode == 'all':
    starttime = datetime.strptime(title_date, '%m/%d/%Y') - timedelta(days=5)
elif runmode == 'update':
    starttime = datetime.strptime(last_run, '%m/%d/%Y %H:%M')
else:
    starttime = datetime.today() - timedelta(days=7)

allorders = allorders[pd.to_datetime(allorders['Date']) > starttime]  

printed = len(allorders.index)
print('{count} orders filtered by date and time.'.format(count = printed))

35 orders filtered by date and time.


In [6]:
# Coalesce columns to simplify the order form.

# Combine first and last name.
allorders.insert(1, 'Name', allorders['First Name'] + ' ' + allorders['Last Name'])

# Format the Zip code correctly.
allorders['Zip'] = allorders['Zip'].astype(str).str.pad(5,fillchar='0')
allorders['Street Address'] = allorders['Street Address'].astype(str)

# Combine address elements into a single address field.
allorders.insert(5, 'Address', allorders['Street Address'] + '<br>' + allorders['City'] + ' ' + allorders['State'] + ' ' + allorders['Zip'])

# Replace "NaN" values with blanks.
allorders = allorders.replace(np.nan, '')

# Drop the fields we combined.
# allorders = allorders.drop(columns=['Date', 'First Name', 'Last Name', 'Street Address', 'City', 'State', 'Zip'])
allorders = allorders.drop(columns=['Date', 'First Name', 'Last Name', 'Street Address', 'City', 'State', 'Zip'])

In [7]:
display(allorders)

,Name,"<strong>*** THANKSGIVING ANNOUNCEMENT ***<br /><br /></strong>The Food Pantry will be <strong>CLOSED</strong> to normal food pantry operations Thanksgiving week (Thursday, November 23rd) due to the special Thanksgiving meal that is being distributed. There will <strong><span style=""text-decoration: underline;"">not</span></strong> be an online order form the week of November 19th. Online ordering will resume on Sunday, November 26th.<br /><br />Shoppers -- If you pre-ordered a frozen turkey, please order any desired side dishes <span style=""text-decoration: underline;"">in this week's food order</span>. Sides will not be distributed with the turkeys.",Address,Number of people in household,Email,Phone,Pickup Time,"Indicate any dietary restrictions (Peanut allergy, low sodium, vegetarian only, etc.):",MEATS/FROZEN ITEMS,REFRIGERATED ITEMS,...,PANTRY - Canned Meats,PANTRY - Canned Vegetables,PANTRY - Canned Beans,PANTRY - Beverages,PANTRY - Breakfast items,PANTRY - Baking/Cooking items,PANTRY - Condiments,PANTRY - Snacks/Other,Special Requests/Notes for Non-Perishable Food Items:,Anything else we should know?
0,Farren Lydston,,184 GREAT BAY WOODS<br>NEWMARKET NH 03857,4,farren630@gmail.com,6038151755,Delivery - must be on pre-approved list,,Beef Roast/Steak<br />Beef Stew Meat<br />Pork...,Milk<br />Shredded Cheese<br />Butter<br />Yog...,...,,,,Grape Juice,Cereal,Pan Coating Veg. Oil Spray,Ketchup<br />Honey Mustard,Walnuts<br />Raisins<br />Saltines/Crackers<br...,,
1,Camryn Battiparano,,3 rock st<br>Newmarket NH 03857,2,emergencie@gmail.com,6039693291,"Thursday, 12 - 2pm",Gluten free,Turkey Bacon,Yogurt<br />Premade Meal/Side,...,,,,,,Pan Coating Veg. Oil Spray,,Granola Bars<br />Chips/Snacks<br />Dessert/Candy,,
2,Arnold Bonner,,2F Salmon st apt 17<br>Newmarket NH 03857,2,aab02.albj68@gmail.com,6038129927,"Thursday, 12 - 2pm",,"Ground Turkey<br />Boneless Chicken (Breasts, ...",Milk<br />Eggs<br />Deli Sandwich/Wrap<br />Pr...,...,Tuna Fish,,Baked Beans,,Cereal<br />Flavored Instant Oatmeal Packets,,,,,
3,Joelle Ruby Ryan,,5 Pulaski Dr Apt 419<br>Newmarket NH 03857,1,joelle.ruby.ryan@gmail.com,603-767-0726,Delivery - must be on pre-approved list,,Frozen Sliced Peaches,Milk<br />Sliced Cheese<br />Shredded Cheese<b...,...,Canned Chicken,,Baked Beans,Grape Juice<br />Black Tea<br />Herbal Tea,,,Honey Mustard,Chips/Snacks<br />Dessert/Candy,do you ever get soda?,
4,Helen Jones,,299 Wadleigh Falls Rd bldg 1 Apt 102<br>Newmar...,1,Helvis5@yahoo.com,603-978-5257,"Thursday, 4 - 6pm",,,Shredded Cheese<br />Yogurt<br />Deli Meat<br ...,...,,Yams,,,Cereal<br />Flavored Instant Oatmeal Packets,Evaporated Milk,,Granola Bars<br />Chips/Snacks,Joe S will be picking up my order,
5,Joanne Stevens,,28 Gordon Ave. Zone 5<br>Newmarket NH 03857,1,,603-534-3055,Delivery - must be on pre-approved list,,Hotdogs<br />Breakfast Sausage,Shredded Cheese<br />Butter<br />Yogurt<br />C...,...,,,,,Flavored Instant Oatmeal Packets,Pan Coating Veg. Oil Spray,,Saltines/Crackers<br />Granola Bars<br />Chips...,Apple Juice ..,BIRTHDAY IS 11/22 — Birthday Cake??
6,Marci Linscott,,20 Lamprey St. Zone #1<br>Newmarket NH 03857,1,,603-948-6338,Delivery - must be on pre-approved list,,Pizza Dough,Milk<br />Eggs<br />Shredded Cheese<br />Butte...,...,Tuna Fish,Mixed Veggies<br />Potatoes,,Coffee,Flavored Instant Oatmeal Packets,Pan Coating Veg. Oil Spray,,Raisins<br />Saltines/Crackers<br />Chips/Snac...,cracks,paper towels
7,Tami Springer,,43B Packer Falls Rd. zone #3<br>Newmarket NH 0...,2,,603-395-0013,Delivery - must be on pre-approved list,,,Yogurt,...,Canned Chicken,Yams,,Chocolate Milk,,,Honey Mustard,,,4 kids
8,Bob Scott,,2 CHAPEL ST APT C <br>Newmarket NH 03857,1,,603-292-5771,Delivery - must be on pre-approved list,,"Boneless Chicken (Breasts, etc.)<br />Ground B...",Milk<br />Shredded Cheese<br />Butter<br />Yog...,...,Tuna Fish,Potatoes<br />Yams,,Chocolate Milk<br />Grape Juice<br />Coffee,Cereal,Sugar<br />Pan Coating Veg. Oil Spray,Ketchup<br 

In [39]:
# Check in https://yoursubdomain.breezechms.com/api/events/attendance/add
# The Python wrapper suggested in https://app.breezechms.com/api seems to have major problems and no support.
# I'm trying https://pypi.org/project/breeze-chms-api/
from breeze_chms_api import breeze

# Initialize API 
breeze_api = breeze.breeze_api(breeze_url='https://newmarketchurch.breezechms.com',
                               api_key='8dfd0a0d7f5aaec745a73542f58eb8ba')

# Get all the people
people = breeze_api.list_people()
display(len(people))

# breeze_api.event_check_in()

# Find the shopping event by name and date.
events = breeze_api.list_events(start=title_date, end=title_date)
event = [e for e in events if e['name'] == 'Food Pantry'][0]
# display(event)

# Get the checked-in shoppers for the event.
shoppers = breeze_api.list_attendance(instance_id=event['id'])
# display(shoppers)
# https://yoursubdomain.breezechms.com/api/events/attendance/list?instance_id=1521321&type=person

1822

In [72]:
online_orders = breeze_api.list_form_entries(form_id='557986')
# display(online_orders)
# for order in online_orders.values():
#     print(order['person_id'])
shopper_ids = [order['person_id'] for order in online_orders.values()]
display(shopper_ids[0])

check = breeze_api.event_check_in(person_id=shopper_ids[0], instance_id=event)

'30396820'

In [9]:
# This cell was run once to save the orders for recurring orders.
# Edit as appropriate and rerun to update the recurring orders.

# recurring = allorders[allorders['Name'] == 'Rosa Soto']
# recurring = recurring.drop(recurring.columns[1], axis=1)
# recurring.to_pickle('recurring.pk1')


In [10]:
# Add recurring orders - orders that have been saved and need to be filled even without a current order form.
allorders = pd.concat([allorders, pd.read_pickle('recurring.pk1')])

In [11]:
# Aggregate multiple orders from the same name.

duporderers = set()

def concatenate(ser):
    # Collect all the requests from all the orders and include each one just once.
    br = '<br />'    # Order request seperator
    contents = set()
    if len(ser) > 1:
        # Keep track of the names on multiple orders so they can be included on the cover sheet. 
        duporderers.add(allorders['Name'].loc[ser.index[0]])
    for elem in ser:
        contents = contents.union(set((str(elem).split(br))))
    contents.discard('')   # Don't keep blank requests.
    return br.join(list(contents))
        
allorders = allorders.groupby('Name', as_index=False).aggregate(concatenate)

allorders = allorders.sort_values(by = 'Pickup Time')

deduped = len(allorders.index)
print('{count} orders deduped.'.format(count = deduped))
if len(duporderers) > 0:
    print('Duplicate orders receivd from {people}'.format(people = duporderers))

35 orders deduped.
Duplicate orders receivd from {'Rosa Soto'}


In [12]:
# Collect the summary for only refrigerated items. ("page 1")
# Include only the following fields in the Summary.
summary = allorders[[
    'Name', 
    'Number of people in household', 
    'Pickup Time',
    'Address', 
    'Email', 
    'Phone', 
    'MEATS/FROZEN ITEMS', 
    'REFRIGERATED ITEMS', 
]]
# summary = summary.sort_values(by = 'Pickup Time') 

In [13]:
def formatshoppers(date, data):
    output = ''
    
    rowvars = {
        'key' : None,
        'val' : None
    }
    for _, row in data.iterrows():
        rowtext = ''
        # Format each field as a row in a table.
        for i in range(len(data.columns)):
            rowvars['key'] = data.columns[i]
            rowvars['val'] = row.iloc[i]
            rowtext += rowhtml.render(rowvars)
        
        # Create a page using the above table.
        output += shopperhtml.render({'date': date, 'data': rowtext, 'shoppername': row['Name']})
    
    return output

In [14]:
from jinja2 import Template

# Define HTML templates using Jinja2 for printing the data.

# Template for the whole report, including style.

orders = Template('''
<!DOCTYPE html>
<style>
    h1 { 
            font-family: Arial;
        }
    table tr td { font-family: Arial; font-size: 5mm; }
    table { width: 100%; }
    th { font-family: Arial; font-size: 7mm; }
    td { border-bottom: 1px solid #ddd; }
    td.category { width: 35%; }
</style>
<html>
<head>
<meta charset="ISO-8859-1">
<title>{{title}}</title>
</head>
<body>
{{body}}
</body>
</html>
''')

# Template for each shopper
# TODO: Should we make the title more useful? 
#       Different titles for main page and summary page?
shopperhtml = Template('''
<h1 style="page-break-before:right;" class="shopper-name">PANTRY ORDER FORM ({{date}})</h1>
<table>
<thead><tr><th colspan=2>{{shoppername}}</th></tr></thead>
{{data}}
</table>
''')

coversheet = Template('''
<h2>NCC Pantry Order Print Cover Sheet for {{date}}</h2>
<dl>
<dt>Total number of orders in this print file</dt>
<dd><b>{{deduped}}</b></dd>
<dt>Order forms in input file (before removing old and duplicate orders)</dt>
<dd>{{received}}</dd>
<dt>Printing orders since</dt>
<dd>{{starttime}}</dd>
<dt>Report run at</dt>
<dd>{{runtime}}</dd>
<dt>Input File</dt>
<dd>{{input_file}}</dd>
<dt>Output File</dt>
<dd>{{output_file}}</dd>
{% for duper in dupers %}
    <dt>Multiple Orders received from</dt>
    <dd>{{duper}}</dd>
{% endfor %}
</dl>

''')

sectionheader = '''
<h1 style="page-break-before:always;">One Page Summaries for Refrigerated Items</h1>
'''

# Template for each row
rowhtml = Template('<tr><td class="category">{{key}}</td><td>{{val}}</td></tr>')


In [15]:
# Build the report using Jinja2.

# Cover Sheet
output = coversheet.render({'received': received, 
                            'printed': printed, 
                            'deduped': deduped, 
                            'date': title_date,
                            'input_file': weekly_orders,
                            'output_file': upload_file, 
                            'dupers': duporderers,
                            'starttime': starttime,
                            'runtime': current_run.strftime("%Y-%m-%d %H:%M:%S"),
                           })

# Full Orders
output += formatshoppers(title_date, allorders) 

# Separator
output += sectionheader

# Refrigerated items
output += formatshoppers(title_date, summary)

html = orders.render({'title': name, 'body': output})

In [16]:
# Write out the file to print.
with open(upload_file, 'w') as f:
    f.write(html)
print('Wrote {file}'.format(file = upload_file))

Wrote C:\Users\ralph\Downloads\NCC Food Pantry Orders.html


In [17]:
from jinja2 import Template

# Define HTML templates using Jinja2 for printing labels

# Template for the address labels, including style.
# Styled to fit Avery 8163 (2" x 4") labels. https://www.avery.com/help/article/avery-labels-2-inch-x-4-inch

labels = Template('''<!DOCTYPE html>
<style>
    @media print {
       @page {
        margin-top: 0.45in;
        margin-bottom: 0.45in;
        margin-left: 0.06in;
        margin-right: 0.06in;    
        }
    }
.label {
  box-sizing: border-box;
  width: 4.0in;
  height: 2.0in;
  padding-top: 0.0in;
  padding-right: 0.25in;
  padding-bottom: 0.125in;
  padding-left: 0.375in;
  border: 0.0in solid black;
  margin-left: 0.1in;
  margin-right: 0.0in;
  margin-top: 0.0in;
  margin-bottom: 0.0in;
  float: left;
  font-family: Arial;
  font-size: 1.0em;
  text-align: left;
  overflow: hidden;
  outline: 0px white;
  page-break-inside: avoid;
}
.name {
  font-size: 1.5em;
}
</style>
<html>
<head>
<meta charset="ISO-8859-1"/>
</head>
<body>{{body}}</body>
</html>''')

# Template for each shopper
# TODO: Should we make the title more useful? 
#       Different titles for main page and summary page?
shopperlabel = Template('''
<div class="label">
<p class='name'>{{shoppername}}</p>
<p>{{pickup}}</p>
<p>{{address}}</p>
<p>{{phone}}</p>
</div>
''')



In [18]:
# Create labels to be attached to the bags for the orders.

output = ''
# Manually add Jay
output += number_of_labels * shopperlabel.render({
        'shoppername': 'Jay Stillman', 
        'pickup': 'Thursday, 12 - 2pm', 
        'address': ' ', 
        'phone': '659-4911'
})

for _, row in summary.iterrows():
    output += number_of_labels * shopperlabel.render({
        'shoppername': row['Name'], 
        'pickup': row['Pickup Time'], 
        'address': row['Address'], 
        'phone': row['Phone']
    })
    


html = labels.render({'body': output})

In [19]:
# Write out the file to print.
with open(label_file, 'w') as f:
    f.write(html)
print('Wrote {file}'.format(file = label_file))

Wrote C:\Users\ralph\Downloads\NCC Food Pantry Orders labels.html


In [20]:

# Requires https://wkhtmltopdf.org/index.html
# Added to path (C:\Program Files\wkhtmltopdf\bin) to environment via virtualenv activate.bat ("C:\Users\ralph\Envs\breeze\Scripts\activate")
# Maybe just
#    pip install git+https://github.com/jontsai/python3-wkhtmltopdf.git
import pdfkit

pdfkit.from_file(upload_file, os.path.splitext(upload_file)[0]+'.pdf')


# Haven't figured out how to get pdfkit to format the labels correctly.
# For now, load the html file in a browser and print to PDF.
pdf_options = {
    'page-size': 'Letter',
    'margin-top': '1.45',
    'margin-right': '0.06',
    'margin-bottom': '0.45',
    'margin-left': '0.06',
}

# pdfkit.from_file(label_file, os.path.splitext(label_file)[0]+'.pdf', options=pdf_options)


In [21]:
# Optional: Display the report here.
import IPython
# IPython.display.HTML(html)